# 1. Introduction

## 1.1 Premise

A manager at a bank is concerned that more and more customers are leaving the bank's credit card services. 

The bank would really appreciate it if someone could help it predict who is going to churn, so that it can proactively approach such customers to offer better services, and turn them back.

## 1.2 Plan

- Perform **exploratory data analysis** to learn the *properties/characteristics* of the features present.
- Fit several **classification models** to predict whether a customer will churn or not.
- Apply *hyper-parameter optimization* techniques.
- **Evaluate performance** of fitted models.

In [ ]:
import numpy as np 
import pandas as pd
from scipy import stats

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.rc('font', family='serif')

# Preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, QuantileTransformer, StandardScaler

# Modelling
from shutil import rmtree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from tempfile import mkdtemp
from xgboost import XGBClassifier

In [ ]:
# Load the data
data = pd.read_csv(
    '/kaggle/input/credit-card-customers/BankChurners.csv',
    index_col='CLIENTNUM',
    na_values=['Unknown']  # interpret "Unknown" as missing 
)
data.head()

# 2. Exploratory Data Analysis

The dataset consists of 10,127 rows and 20 columns.

In [ ]:
# Drop the last 2 columns as advised in the dataset's description
# See https://www.kaggle.com/sakshigoyal7/credit-card-customers
data = data.iloc[:, :-2]
data.info()

## 2.1 Missing Values

`Education_Level`, `Marital_Status` and `Income_Category` have missing values. 

In [ ]:
# Check for missing values
missing = data.isna().sum()
pd.DataFrame({
    'No. of missing values': missing,
    '% missing': missing.apply(lambda x: f'{x/len(data):.2%}')
}).style.bar()

### Strategy for Handling Missing Values

A common and straight-forward way of dealing with missing values is to *drop affected rows or columns*. The advantage of this is that you'll be left with genuine, unaltered data. The disadvantage is that you lose some data; which is especially undesirable if the dataset is small, or large proportions of its values are missing.

Another common tactic is *imputation*, which involves determining values to fill in the blanks. The advantage here is that no data is thrown out. But then, depending on the method used, the imputed values might be misleading.

Removing rows with missing values would in this case leave only 7,081 rows for modelling. That is rather small, so we'll use imputation to get as much of the data as possible. This will be implemented as a component in the model fitting pipeline.

## 2.2 Numeric Features

There are 14 numeric features.

In [ ]:
numeric_cols = data.select_dtypes(include="number")
numeric_cols.describe()

### 2.2.1 Histograms

In [ ]:
# Plot histograms of the numeric columns
histograms = numeric_cols.hist(figsize=(14, 12))

>`Credit_Limit`, `Avg_Open_To_Buy`, `Total_Amt_Chng_Q4_Q1`, `Total_Trans_Amt`, `Total_Ct_Chng_Q4_Q1` and `Avg_Utilization_Ratio` are skewed to the right (positively skewed).

>`Total_Revolving_Bal` has a curious peak close to the origin, which is investigated below:

In [ ]:
total_rev_bal = data['Total_Revolving_Bal'].value_counts()
print(
    f'A very large number of customers ({total_rev_bal[0]:,}) have '
    '"Total_Revolving_Bal" == 0.',
    total_rev_bal.nlargest(5),  # top 5 frequencies
    '\nFrequencies of the first 5 values confirm that the peak is specifically at 0:',
    total_rev_bal.sort_index().head(),
    sep="\n"
)

### 2.2.2 Box-plots

In [ ]:
_ = numeric_cols.plot(kind="box", subplots=True, layout=(5, 3), figsize=(12, 20))

> `Credit_Limit`, `Avg_Open_To_Buy`, `Total_Amt_Chng_Q4_Q1`, `Total_Trans_Amt` and `Total_Ct_Chng_Q4_Q1` have a very large number of outliers.
>
> The `QuantileTransformer` can be used to normalize them.

### 2.2.3 Normal Probability Plots

In [ ]:
# Plot probability plots (qq-plots)
fig, axes = plt.subplots(nrows=5, ncols=3, figsize=(12, 18))

fig.tight_layout(h_pad=5)  # Add padding to sub-plots

for col, ax in zip(numeric_cols.columns, axes.flatten()):
    stats.probplot(numeric_cols[col], dist='norm', plot=ax)
    ax.set_title(col)

fig.delaxes(axes[-1, -1])  # Remove 15th (last) axes which has no content

- `Customer_Age`, `Dependent_count`, `Months_on_book` and `Total_Trans_Ct` are somewhat normally distributed, which is good.

## 2.3 Categorical Features

There are 5 categorical features. The target variable `Attrition_Flag` is also categorical.

In [ ]:
categorical_cols = data.select_dtypes(include='O')
categorical_cols.describe()

### 2.3.1 Count-plots

In [ ]:
# Plot countplots of categorical columns
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 14))

for col, ax in zip(categorical_cols.drop('Attrition_Flag', 1), axes.flatten()):
    sns.countplot(data=categorical_cols, x=col, hue='Attrition_Flag', ax=ax)
    ax.tick_params(axis='x', rotation=45)
    ax.set_title(col, size=14)
    
fig.delaxes(axes[-1, -1])  # Drop 6th (last) axes as it has no content
plt.tight_layout()

# 3. Modelling & Prediction

Let's now attempt to fit several classification models to predict whether a customer will leave.

The target variable - `Attrition_Flag` - is heavily imbalanced, with one class having significantly higher occurences than the rest.

In [ ]:
_ = sns.countplot(x=categorical_cols['Attrition_Flag'])

> There exist strategies for handling such situations, some of which can be implemented using the [imbalanced-learn][1] package.

> In this case, we'll use [Randomized over-sampling][2].

[1]: https://imbalanced-learn.org/stable/user_guide.html
[2]: https://imbalanced-learn.org/stable/over_sampling.html#naive-random-over-sampling

In [ ]:
# Select the features and target
X = data.drop('Attrition_Flag', axis=1)
y = data['Attrition_Flag']

# Prepare a training and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


# Preprocessing pipeline for numeric cols
numeric_pipe = Pipeline([
    ('normalize', QuantileTransformer(output_distribution='normal')),
    ('rescale', StandardScaler())
])

# Preprocessing pipeline for categorical cols
categorical_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('encode', OneHotEncoder())
])

# Combined preprocessing pipeline
extract_features = ColumnTransformer([
    ('numeric', numeric_pipe, numeric_cols.columns),
    ('categorical', categorical_pipe, categorical_cols.columns.drop('Attrition_Flag'))
])

# Apply random oversampling to training data to counteract target class imbalance
random_oversampler = RandomOverSampler(random_state=0)
X_resampled, y_resampled = random_oversampler.fit_resample(X_train, y_train)


def fit_and_evaluate(classifier, params=None):
    """Fit a classification model and print metrics.
    
    Parameters
    ----------
    classifier: sklearn predictor instance
        The model to fit.
    params: dict
        A dictionary of hyper-parameter values to pass to GridSearchCV 
        for model tuning.
    
    Returns
    -------
    sklearn predictor
        The model with hyper-parameters yielding the highest cross-validated score.
    """
    cache_dir = mkdtemp()
    pipe = Pipeline(
        [('features', extract_features), ('classifier', classifier)],
        memory=cache_dir
    )
    
    params = params or {} # use empty dict if params not provided
    
    model = GridSearchCV(estimator=pipe, param_grid=params, scoring='roc_auc',
                         n_jobs=2, cv=4, verbose=1)
    model.fit(X_resampled, y_resampled)

    rmtree(cache_dir)
    print(
        f'\nCross Validation AUC: {model.best_score_:.4f}',
        '\nClassification Report (on test data):\n' + '-'*58,
        classification_report(model.predict(X_test), y_test),
        sep='\n'
    )
    
    return model.best_estimator_

## 3.1 Random Forest Classifier

In [ ]:
clf = RandomForestClassifier(random_state=7)
params = {'classifier__max_depth': range(3, 8),
          'classifier__n_estimators': [100, 200],
          'classifier__class_weight': ["balanced", "balanced_subsample"]}

rf_model = fit_and_evaluate(clf, params)

## 3.2 Gradient Boosting Classifier

In [ ]:
clf = XGBClassifier(learning_rate=0.02, random_state=0, subsample=0.8)
params = {'classifier__max_depth': range(3, 7),
          'classifier__reg_lambda': np.logspace(0, 3, 4),
          'classifier__n_estimators': [100, 250]}

gb_model = fit_and_evaluate(clf, params)

## 3.3 Support Vector Classifier

In [ ]:
clf = SVC(class_weight='balanced', random_state=0)
params = {'classifier__C': np.logspace(0, 4, 10)}

svc_model = fit_and_evaluate(clf, params)

## 3.4 Logistic Regression

In [ ]:
clf = LogisticRegression(random_state=2, class_weight='balanced')
params = {'classifier__C': np.logspace(0, 4, 10)}

lr_model = fit_and_evaluate(clf, params)

# 4. Conclusion

Among the models tested above, the `XGBClassifier` seems the most promising.

Let's visualise sample predictions to check if the predictions from the various models are consistent:

In [ ]:
sample = X.sample(25, random_state=5)

results = pd.DataFrame({
    'Random Forest Classifier': rf_model.predict(sample),
    'Gradient Boosting Classifier': gb_model.predict(sample),
    'Support Vector Classifier': svc_model.predict(sample),
    'Logistic Regression': lr_model.predict(sample)
})


def color_code(cell):
    """Set a DataFrame cell's background color according to its value."""
    if cell == 'Existing Customer':
        color = 'aqua'
    elif cell == 'Attrited Customer':
        color = 'orangered'
    return f'background-color: {color}'

results.style.applymap(color_code)